## Import necessary packages

In [1]:
from subprocess import call
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shutil import copyfile, move, copy, copytree, rmtree
from distutils.dir_util import copy_tree
from IPython.display import clear_output
from tempfile import mkstemp
import copy
from os import fdopen, remove

from ema_workbench import (RealParameter, CategoricalParameter, IntegerParameter, BooleanParameter, ScalarOutcome, ArrayOutcome, Constant, Model, MultiprocessingEvaluator, Policy, perform_experiments, ema_logging )
from ema_workbench import (save_results, load_results, experiments_to_scenarios)
from ema_workbench.em_framework import (sample_uncertainties)

from functools import partial
from multiprocessing import Pool, Lock
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.pyplot as plt
from scipy import stats

from numpy.lib import recfunctions as rf

import pickle

C:\Users\Admin_2\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Set directory

In [2]:
os.chdir("C:/Users/Admin_2/Documents/Thesis/LUS/DeltaScenarios_TUdelft1")
notebook_dir = os.getcwd()


## Link to GeoDMS software

In [3]:
#Function to run GeoDMS:
def geodmsrun(config, tree, geodms_dir = r"C:/Program Files/ObjectVision/GeoDms7212"): #change this to your GeoDms directory
    
    #change working directory
    notebook_dir = os.getcwd()
    os.chdir(geodms_dir)
    assert os.path.isdir(geodms_dir)
    
    print(os.getcwd())
    
    #run the geodms
    arg_exe = ['GeoDmsRun.exe', config, tree]
    call(arg_exe)
    
    #change back the working directory
    os.chdir(notebook_dir)
    print(os.getcwd())
    
def find_line(file_path, pattern):
    #Create temp file
    lines = []
    fh, abs_path = mkstemp()
    with fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                if pattern in line:
                    lines.append(line)
    return lines

#Function which will determine the new line containing the new value:
def new_line(beginline, line, new_val, endline):
    newline = beginline + str(new_val) + line[line.find(endline):]
    return newline

#Function which actually replaces the value of demand
def replace_line(beginline, file_path, pattern, new_val, endline):
    #Create temp file
    fh, abs_path = mkstemp()
    with fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                if line==pattern:
                    subst = new_line(beginline, line, new_val, endline)
                    new_file.write(line.replace(pattern, subst))
                else:
                    new_file.write(line)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)


## Changing the land use claims

### Residential claim parameters
Residential claim
To be able to switch between de input of housing demand for each scenario, a few steps are taken:

Four files are created in the source data (SD/claims), which each contain different housing demand generated by the TigrisXL model. These files are:

- TigrisXL (hoog_spreiding)
- TigrisXL2 (hoog_concentratie)
- TigrisXL3 (laag_spreiding)
- TigrisXL4 (laag_concentratie)

Within these four files, the file which contains the housing demand per year is, in all four TigrisXL files, named "hoog_spreiding". This is done because we use the STOOM scenario as our default model run, which will only recognize files named "hoog_spreiding".

Next, the actual housing demand in Ha is calculated using a density factor (DensityFactor) and an "inbreidingsfractie". So for each scenario, the corresponding values of these factors are:

- hoog_spreiding: DensityFactor = 1.0, Inbreidingsfractie = 0.25
- hoog_concentratie: DensityFactor = 1.33, Inbreidingsfractie = 0.75
- laag_spreiding: DensityFactor = 1.0, Inbreidingsfractie = 0.25
- laag_concentratie: DensityFactor = 1.33, Inbreidingsfractie = 0.75

So, to change the housing claim input, we will run with the different TigrisXL files and using different density factors and inbreidingsfractie.

In [4]:
def changeresidentialclaim(TigrisXLfile, inbreiding, Density):
    if TigrisXLfile == 1:
        TLXfilename = 'TigrisXL'
    elif TigrisXLfile == 2:
        TLXfilename = 'TigrisXL2'
    elif TigrisXLfile == 3:
        TLXfilename = 'TigrisXL3'
    elif TigrisXLfile == 4:
        TLXfilename = 'TigrisXL4'
    #Change TigrisXL file in claimswonen.dms and claimspopulation.dms
    claimswonendms = notebook_dir+'/PD/cfg/stam/Templates/ClaimsWonen.dms'
    claimspopulationdms = notebook_dir+'/PD/cfg/stam/Templates/ClaimsPopulation.dms'
    houseclaimsearch = '%SourceDataProjDir%/Claims/Tigris'
    housebeginline = '\t\t:\tStorageName = "=\'%SourceDataProjDir%/Claims/'
    houseendline = '/\'+ ClaimSrcDir + \'/landuse/landuse.dat\'"\n'
    popbeginline = '\t\t,\tStorageName = "=\'%SourceDataProjDir%/Claims/'
    popendline ='/\'+ dirname + \'/\' + jaar + \'/segs/segs.dbf\'"\n'
    
    houseline = find_line(claimswonendms,houseclaimsearch)[0]
    popline = find_line(claimspopulationdms, houseclaimsearch)[0]
    
    
    #Change densityfactor and Infringement fraction in scenarios.dms
    if inbreiding == True:
        inbreidingvalue = 0.25
    else:
        inbreidingvalue = 0.75
    
    scenariosdms = notebook_dir+'/PD/cfg/stam/Classifications/Scenarios.dms'
    
    inbreidingclaimsearch = 'attribute<float32> Inbreidingsfractie'
    inbreidingbeginline = '\t\t\tattribute<float32> Inbreidingsfractie : ['
    inbreidingendline = ', 0.75]'
    
    inbreidingline = find_line(scenariosdms, inbreidingclaimsearch)[0]
    replace_line(inbreidingbeginline, scenariosdms, inbreidingline, inbreidingvalue, inbreidingendline)
    
    if Density == True:
        densityvalue = 1.33
    else:
        densityvalue = 1.0
    
    densclaimsearch = 'attribute<float32> DensityFactor'
    densline = find_line(scenariosdms, densclaimsearch)[0]
    densbeginline = '\t\t\tattribute<float32> DensityFactor: ['
    densendline = ', 1.33]'
    
    
    replace_line(housebeginline, claimswonendms, houseline, TLXfilename, houseendline)
    replace_line(popbeginline, claimspopulationdms, popline, TLXfilename, popendline)
    replace_line(densbeginline, scenariosdms, densline, densityvalue, densendline)

### Industry & commerce claim parameters
To change the work claim, a few steps have to be taken: 
- Same as for housing claim, the four TigrisXL files are used and can be changed. 
- Next, with this function you can also change the location type preference (LVQ).Lastly, with this function you can also change the terraincoefficient (TYQ). LVQ and TYQ go together and are not to change seperately. For this reason, both variable are determined with "Claimfile". This variable, Claimfile, will determing which file the LVQ and TYQ are read from. 

TigrisXL files (TigrisXLfile):
- See Residential claim 

Claim files (Claimfilename):
- Claims (True) = Hoog scenario
- Claims2 (False) = Laag scenario

In [5]:
def changeworkclaim(TigrisXLfile, Claimfile):
    if TigrisXLfile == 1:
        TLXfilename = 'TigrisXL'
    elif TigrisXLfile == 2:
        TLXfilename = 'TigrisXL2'
    elif TigrisXLfile == 3:
        TLXfilename = 'TigrisXL3'
    elif TigrisXLfile == 4:
        TLXfilename = 'TigrisXL4'
    #Change work claim source data
    claimswerkendms = notebook_dir+'/PD/cfg/stam/Templates/ClaimsWerken.dms'
    workclaimsearch = '%SourceDataProjDir%/Claims/Tigris'
    workline = find_line(claimswerkendms, workclaimsearch)[0]
    workbeginline = '\t\t:\tStorageName = "=\'%SourceDataProjDir%/Claims/'
    workendline = '/\'+ dirname + \'/\' + jaar + \'/labour/labour.dat\'"\n'
    replace_line( workbeginline, claimswerkendms, workline, TLXfilename, workendline)
   
    if Claimfile == True:
        Claimfilename = 'Claims'
    else:
        Claimfilename = 'Claims2'

    #Change LVQ & TYQ files 
    LVQreadsearch = '%SourceDataProjDir%/Claims'
    LVQline = find_line(claimswerkendms,LVQreadsearch )[1]
    LVQbeginline = '\t\t:\tStorageName = "=\'%SourceDataProjDir%/'
    LVQendline = '/\'+ dirname + \'/\'+ jaar + \'/\'+ filename +\'\'"\n'
    
    
    TYQline = find_line(claimswerkendms,LVQreadsearch )[2]
    TYQbeginline = '\t\t:\tStorageName = "=\'%SourceDataProjDir%/'
    TYQendline = '/\'+ dirname + \'/\'+ jaar + \'/\'+ filename +\'\'"\n'
    
    
    replace_line( TYQbeginline, claimswerkendms,  TYQline, Claimfilename, TYQendline)
    replace_line( workbeginline, claimswerkendms, workline, TLXfilename, workendline)
    replace_line( LVQbeginline, claimswerkendms, LVQline, Claimfilename, LVQendline)

### Nature claim parameters
Nature claim values (natureclaim):
- value 1 = 106000 (DRUK)
- value 2 = 75000 (STOOM)
- value 3 = 50000 (WARM & RUST)

In [6]:
def changenatureclaim(natureclaim):
    if natureclaim == 1:
        natureha = 106000
    elif natureclaim == 2:
        natureha = 75000
    elif natureclaim == 3:
        natureha = 50000
    
    Naturedms = notebook_dir+'/PD/cfg/stam/Claims/TotalenInBron.dms' #change to location file
    Natureclaimsearch = 'attribute<claimHa> STOOM  := Nl/ID == 1[uint8] ?' #line of code it will look for to find where parameter should be changed
    naturebeginline = str('\t\t\t\tattribute<claimHa> STOOM  := Nl/ID == 1[uint8] ?  ') #how the line should look before the parameter value
    natureendline = str('[claimHa]') # how the line should look after the parameter value
    natureline = find_line(Naturedms, Natureclaimsearch)[0]
    
    
    replace_line(naturebeginline, Naturedms, natureline, natureha, natureendline ) #replaces the line

### Recreation claim parameters
Recreation claim (recvalue):
- True = GE (hoog scenario)
- False = RC (laag scenario) 

In [7]:
def changerecreationclaim(recvalue):
    if recvalue == True:
        GEorRC = 'GE'
    else:
        GEorRC = 'RC'
    recreationdms = notebook_dir+'/PD/cfg/stam/Claims/TotalenInBron/Verblijfsrecreatie.dms'
    recreationclaimsearch = '%SourceDataProjDir%/Claims/Ruimte/verblijfsrec_'
    recline = find_line(recreationdms, recreationclaimsearch)[0]
    recbeginline = '        ", \'NaarCases(\'+quote(\'%SourceDataProjDir%/Claims/Ruimte/verblijfsrec_'
    recendline = '_provincie'
    
    
    replace_line(recbeginline, recreationdms, recline, GEorRC, recendline)

### Agriculture claim parameters
For all agrclaim values:
- True = hoog scenario
- False = laag scenario


In [8]:
def changeagricultureclaim(agrclaim):
    
    agrdms = notebook_dir+'/PD/cfg/stam/Claims/TotalenInBron/AgrIntens.dms'
    
    if agrclaim == True:
        glastuinbouwdecrease = 0.1
        opengroentenha = 27100
        bloembollenha = 23600
        boomgaardha = 17300
        boomteeltha = 17100
    else:
        glastuinbouwdecrease  = 0.3
        opengroentenha = 22100
        bloembollenha = 21200
        boomgaardha = 15400
        boomteeltha = 15400
        
    agrclaimsearch5 = 'attribute<ggHa> agr_glastuinbouw'
    agrbeginline5 = str('\t\t\tattribute<ggHa> agr_glastuinbouw (NL):= NL/ID == 1[uint8] ? InBasisjaar/In2011/agr_glastuinbouw + -')
    agrendline5 = '[float32]'
    agrline5 = find_line(agrdms, agrclaimsearch5)[1]
    
        
    agrclaimsearch1 = 'attribute<ggHa> agr_OpenGroenten (NL):= NL/ID == 1[uint8] ?'
    agrbeginline1 = str('\t\t\tattribute<ggHa> agr_OpenGroenten (NL):= NL/ID == 1[uint8] ? ')
    agrendline = '[ggHa]'
    agrline1 = find_line(agrdms, agrclaimsearch1)[1]
    
    
    agrclaimsearch2 = 'attribute<ggHa> agr_bloembollen'
    agrbeginline2 = str('\t\t\tattribute<ggHa> agr_bloembollen  (NL):= NL/ID == 1[uint8] ? ')
    agrline2 = find_line(agrdms, agrclaimsearch2)[1]
    
    
    
    agrclaimsearch3 = 'attribute<ggHa> agr_boomgaard'
    agrbeginline3 = str('\t\t\tattribute<ggHa> agr_boomgaard    (NL):= NL/ID == 1[uint8] ? ')
    agrline3 = find_line(agrdms, agrclaimsearch3)[1]
    
    
    
    agrclaimsearch4 = 'attribute<ggHa> agr_boomteelt'
    agrbeginline4 = str('\t\t\tattribute<ggHa> agr_boomteelt    (NL):= NL/ID == 1[uint8] ? ')
    agrline4 = find_line(agrdms, agrclaimsearch4)[1]
    
    replace_line(agrbeginline1, agrdms, agrline1, opengroentenha, agrendline)
    replace_line(agrbeginline5, agrdms, agrline5, glastuinbouwdecrease, agrendline5)
    replace_line(agrbeginline4, agrdms, agrline4, boomteeltha, agrendline)
    replace_line(agrbeginline2, agrdms, agrline2, bloembollenha, agrendline)
    replace_line(agrbeginline3, agrdms, agrline3, boomgaardha, agrendline)

## Changing the suitability

### Nature suitability
For nature location suitability, three possible values exist: 

- Valuenaturelocation 1 (WARM & RUST)
- Valuenaturelocation 2 (STOOM)
- Valuenaturelocation 3 (DRUK)

Next, whether or not nature policy stimuli are in place is defined by the "policyweightnature": 

- Policyweightnature = 5.0 (STOOM, DRUK, RUST)
- Policyweightnature = 0.0 (WARM)

Lastly, the physical suitability for nature depends on whether or not water is pumped: 

- Pumping = True (STOOM, DRUK, RUST)
- Pumping = False (WARM)

In [9]:
def naturesuitability(valuenaturelocation, policyweightnature, pumping):
    #changing the location factor value 
    if valuenaturelocation == 1:
        locationweight = '0.0'
        scenario1 = 'WARM,'
        scenario2 = 'DRUK'
    elif valuenaturelocation == 2: 
        locationweight = 'woongebied/locatie'
        scenario1 = 'STOOM,'
        scenario2 = 'DRUK'
    elif valuenaturelocation == 3:
        locationweight = '0.0'
        scenario1 = 'DRUK,'
        scenario2 = 'STOOM'
        
    suitabilitynaturedms = notebook_dir+'/PD/cfg/stam/Templates/geschiktheid/natuur.dms'
    
    
    #changing the scenario:
    scenario1search = 'Scenario == /ScenarioUnit/V/'
    scenario1line = find_line(suitabilitynaturedms,scenario1search )[0]
    scenario1beginline = '\t\t\t\tScenario == /ScenarioUnit/V/'
    scenario1endline = '\n'
   

    scenario2search = 'Scenario == /ScenarioUnit/V/'
    scenario2line = find_line(suitabilitynaturedms,scenario1search )[1]
    scenario2beginline = '\t\t\t\tScenario == /ScenarioUnit/V/'
    scenario2endline =  ' || Scenario == /ScenarioUnit/V/PARIJS,\n'
    
    
    #changing location factor value weight:
    locationweightsearch = 'value('
    locationweightline = find_line(suitabilitynaturedms,locationweightsearch )[3]
    locationweightbeginline = '\t\t\t\tvalue('
    locationweightendline = ', Eur_M2_jaarlijks)'
    
    replace_line( locationweightbeginline, suitabilitynaturedms, locationweightline, locationweight, locationweightendline)
    replace_line( scenario2beginline, suitabilitynaturedms, scenario2line, scenario2, scenario2endline)
    replace_line( scenario1beginline, suitabilitynaturedms, scenario1line, scenario1, scenario1endline)
    
    if policyweightnature == 1:
        scenario1a = 'STOOM'
        scenario2a = 'DRUK'
        scenario3a = 'RUST'
    elif policyweightnature == 2: 
        scenario1a = 'DRUK'
        scenario2a = 'STOOM'
        scenario3a = 'RUST'
    elif policyweightnature == 3:
        scenario1a = 'RUST'
        scenario2a = 'DRUK'
        scenario3a = 'STOOM'
    elif policyweightnature == 4:
        scenario1a = 'WARM'
        scenario2a = 'DRUK'
        scenario3a = 'RUST'
        
    #changing the scenario:
    scenario1asearch = '//STOOM1a'
    scenario1aline = find_line(suitabilitynaturedms,scenario1asearch)[0]
    scenario1abeginline = '\t\t\t\tScenario == /ScenarioUnit/V/'
    scenario1aendline = ' && //STOOM1a'
   
    scenario2asearch = '//DRUK1a'
    scenario2aline = find_line(suitabilitynaturedms,scenario2asearch)[0]
    scenario2abeginline = '\t\t\t\t(Scenario == /ScenarioUnit/V/'
    scenario2aendline = ' || Scenario == /ScenarioUnit/V/PARIJS) //DRUK1a'
    
    scenario3asearch = '//RUST1a'
    scenario3aline = find_line(suitabilitynaturedms,scenario3asearch)[0]
    scenario3abeginline = '\t\t\t\tScenario == /ScenarioUnit/V/'
    scenario3aendline = ' //RUST1a'

    replace_line(scenario1abeginline, suitabilitynaturedms, scenario1aline, scenario1a,  scenario1aendline)
    replace_line(scenario2abeginline, suitabilitynaturedms, scenario2aline, scenario2a,  scenario2aendline)
    replace_line(scenario3abeginline, suitabilitynaturedms, scenario3aline, scenario3a,  scenario3aendline)
    
    #changing fysical policy model structure:
    if pumping == True:
        pumppolicy = 'Bronnen/endstate/PotentieNat/PotentieNatPercentage'
    else:
        pumppolicy = 'Omgeving/natuurlandschap/NatuurAdaptatie/potentie_nathuidig'
        
    #Changing the pump policy: 
    pumppolicysearch = '//omdat in dit scenario wel wordt gepompt'
    pumppolicyline = find_line(suitabilitynaturedms,pumppolicysearch)[0]
    pumppolicybeginline = '\t\t\t\t\t\t'
    pumppolicyendline =  '//omdat in dit scenario wel wordt gepompt bij bodemdaling\n'
    
    
    
    replace_line( pumppolicybeginline, suitabilitynaturedms, pumppolicyline, pumppolicy, pumppolicyendline)
    
    
    
    
    

### Nature policy

The nature policy also influences the suitability of multiple land use classes. There are three possible nature policies:

- Naturepolicymap 1: "Robuust" and "Vitaal" nature policy (DRUK)
- Naturepolicymap 2: "ehs uitgekleed" nature policy (STOOM)
- Naturepolicymap 3: "Functionele natuur" nature policy (WARM & RUST)

Next to this, nature also poses restrictions on other land use classes, as to protect nature areas. The importance of these restrictions are defined with the "restrictionnatureweight". Two values are possible: 

- Restrictionnatureweightvalue = 30.0 (STOOM, DRUK & RUST)
- Restrictionnatureweightvalue = 0.0 (WARM) 

In [10]:
def changenaturepolicy(naturepolicymap, restrictionnatureweight):
    if naturepolicymap == 1:
        naturepolicy = 'RobuustVitaal_natuur' #or ehs_uitgekleed or functionele_natuur
    elif naturepolicymap == 2:
        naturepolicy = 'ehs_uitgekleed'
    elif naturepolicymap == 3:
        naturepolicy = 'functionele_natuur'
        
    
    #change policy maps nature
    suitabilitynaturedms = notebook_dir+'/PD/cfg/stam/Templates/geschiktheid/natuur.dms'
    policynaturesearch1 = ' //Policymaphere1'
    naturepolicy1line = find_line(suitabilitynaturedms,policynaturesearch1)[0]
    naturepolicy1beginline = '\t\t\t\t\t(Beleid/NatuurLandschap/'
    naturepolicy1endline =  ' >= 0[Classifications/Beleid/ndt_multi] //Policymaphere1'
    
    policynaturesearch2 = ' //Policymaphere2'
    naturepolicy2line = find_line(suitabilitynaturedms,policynaturesearch2)[0]
    naturepolicy2beginline = '\t\t\t\t&& Beleid/NatuurLandschap/'
    naturepolicy2endline =  ' >= 0[Classifications/Beleid/ndt_multi], //Policymaphere2'
   
    policynaturesearch3 = ' //Policymaphere3'
    naturepolicy3line = find_line(suitabilitynaturedms,policynaturesearch3)[0]
    naturepolicy3beginline = '\t\t\t\t&& Beleid/NatuurLandschap/'
    naturepolicy3endline =  ' >= 0[Classifications/Beleid/ndt_multi], //Policymaphere3'
    
    replace_line(naturepolicy1beginline, suitabilitynaturedms, naturepolicy1line, naturepolicy,  naturepolicy1endline)
    replace_line(naturepolicy2beginline, suitabilitynaturedms, naturepolicy2line, naturepolicy,  naturepolicy2endline)
    replace_line(naturepolicy3beginline, suitabilitynaturedms, naturepolicy3line, naturepolicy,  naturepolicy3endline)
    
    policyrestrictionsdms = notebook_dir+'/PD/cfg/stam/Templates/geschiktheid/beleid_restricties.dms'
    restrictionnaturepolicysearch = '"&& Beleid/NatuurLandschap/'
    restrnaturepolicyline = find_line(policyrestrictionsdms,restrictionnaturepolicysearch)[0]
    restrnaturepolicybeginline = '\t\t\t\t"&& Beleid/NatuurLandschap/'
    restrnaturepolicyendline =  ' >= 0[Classifications/Beleid/ndt_multi]"\n'
    
    
    
    #change wieght policy restriction
    if restrictionnatureweight == True:
        restrictionnatureweightvalue = 30.0
    else:
        restrictionnatureweightvalue = 0.0
        
    restrictionnaturesearch = '//onlythisone'
    restrictionnatureline = find_line(policyrestrictionsdms,restrictionnaturesearch)[0]
    restrictionnaturebeginline = '\t\t\t\t",  value('
    restrictionnatureendline = ', Eur_M2_jaarlijks)"//onlythisone'
    
    replace_line(restrnaturepolicybeginline, policyrestrictionsdms, restrnaturepolicyline, naturepolicy, restrnaturepolicyendline)
    replace_line(restrictionnaturebeginline, policyrestrictionsdms, restrictionnatureline, restrictionnatureweightvalue,  restrictionnatureendline)

### Residential suitability 

The suitability of an area for residential land use depends on the preference of society. More specifically, it depends on whether people rather live within existing residential areas (concentrated) or outside of existing residential areas (scattered). This is defined with the so-calles "spreidingratio", which translates to scatter ratio. Two values are possible: 

- spreidingratio = 0.5 (STOOM & RUST)
- spreidingratio = 1.0 (DRUK & WARM) 

In [11]:
def changehousingsuitability(spreidingratio):
    if spreidingratio == True: 
        ratio = 0.5
    else:
        ratio = 1.0
    #Changing the spread ratio:
    housingsuitabilitydms = notebook_dir+'/PD/cfg/stam/Templates/geschiktheid/woongebied.dms'
    spreidingratiosearch = 'iif(Spreiding'
    spreidingratioline = find_line(housingsuitabilitydms,spreidingratiosearch)[0]
    spreidingratiobeginline = '\t\t* iif(Spreiding, value('
    spreidingratioendline = ', Ratio), value(1.0, Ratio))\n'
    
    
    replace_line( spreidingratiobeginline, housingsuitabilitydms,  spreidingratioline, ratio, spreidingratioendline) 

## Model definition

This section, the model is defined. Here, the relevant functions, the link to GeoDMS, and model outcomes are defined.
The model result we are interested in, is the land use map. However, the EMA workbench can only function with at least two outcome variables. For this reason, a dummy variable is defined as "dummy", which has no further use in this research. 

In [12]:
def LUS_model(TigrisXLfile, inbreiding, Density, Claimfile, natureclaim, recvalue, restrictionnatureweight, naturepolicymap, policyweightnature, spreidingratio, valuenaturelocation, pumping, agrclaim):
    
    changeresidentialclaim(TigrisXLfile, inbreiding, Density)
    changeworkclaim(TigrisXLfile, Claimfile)
    changenatureclaim(natureclaim)
    changerecreationclaim(recvalue)
    changeagricultureclaim(agrclaim)
    
    naturesuitability(valuenaturelocation, policyweightnature, pumping)
    changehousingsuitability(spreidingratio)
    changenaturepolicy(naturepolicymap, restrictionnatureweight)
  
    
    
    #Run GeoDMS software:
    config = notebook_dir+'/PD/cfg/stam.dms'
    tree = '/EndResults/PerScenTimeUnit/STOOM_Y2012_Y2050/Run1'

    geodmsrun(config, tree)
    
    # Safe resulting land use map:
    src = notebook_dir+'/PD/cfg/stam/results/run1.asc'
    
    # Define output: 
    lusmap = np.loadtxt(src, skiprows=6)
    
    # Define dummy output:
    dummy = np.max(lusmap)
    
    return lusmap, dummy

### Experimental set-up

This section of code defines the experimental set-up. For this research, only Boolean parameters and Categorical parameters are used. The outcome is defined as an Array outcome, since the resulting land use maps are stored as arrays. 

In [13]:
ema_logging.log_to_stderr(ema_logging.INFO)
    
LUSmodel = Model('LUS', function=LUS_model)
    
LUSmodel.uncertainties = [  BooleanParameter('agrclaim'),
                                CategoricalParameter('TigrisXLfile', (1,2,3,4)),
                                BooleanParameter('inbreiding'),
                                BooleanParameter('Density'),
                                BooleanParameter('Claimfile'),
                                CategoricalParameter('natureclaim', (1,2,3)),
                                BooleanParameter('recvalue'),
                                BooleanParameter('restrictionnatureweight'),
                                CategoricalParameter('naturepolicymap', (1,2,3)),
                                CategoricalParameter('policyweightnature', (1,2,3,4)),
                                BooleanParameter('spreidingratio'),
                                CategoricalParameter('valuenaturelocation', (1,2,3)),
                                BooleanParameter('pumping')]
                                    
                               
                             

LUSmodel.outcomes = [ArrayOutcome('lusmap')]
    

In [14]:
scenarios2000 = pickle.load( open("C:/Users/Admin_2/Documents/Thesis/LUS/DeltaScenarios_TUdelft1/BramkaAttempt2/2000experimentsslice2MARGRIET.p", "rb" )) 

In [28]:
scenarios2000.designs[350:700]

[(0.0, 0.0, 3.0, 1.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0),
 (0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0),
 (1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 0.0, 0.0, 1.0, 0.0, 2.0),
 (1.0, 0.0, 3.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0),
 (1.0, 0.0, 0.0, 0.0, 1.0, 2.0, 2.0, 0.0, 1.0, 1.0, 1.0, 0.0, 2.0),
 (0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0),
 (0.0, 0.0, 3.0, 0.0, 0.0, 1.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0),
 (1.0, 1.0, 2.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0),
 (0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 2.0),
 (0.0, 0.0, 2.0, 0.0, 1.0, 1.0, 2.0, 2.0, 1.0, 0.0, 1.0, 0.0, 1.0),
 (1.0, 1.0, 1.0, 0.0, 0.0, 2.0, 2.0, 3.0, 0.0, 0.0, 1.0, 1.0, 0.0),
 (0.0, 1.0, 2.0, 0.0, 1.0, 2.0, 0.0, 3.0, 1.0, 1.0, 1.0, 0.0, 1.0),
 (0.0, 1.0, 3.0, 0.0, 1.0, 2.0, 1.0, 3.0, 1.0, 1.0, 0.0, 0.0, 2.0),
 (1.0, 1.0, 3.0, 1.0, 1.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 1.0),
 (1.0, 1.0, 3.0, 1.0, 1.0, 2.0, 0.0, 2.0, 0.0, 0

In [17]:
margrietslice2a = copy.deepcopy(scenarios2000)
margrietslice2a.designs = margrietslice2a.designs[0:350]
margrietslice2a.n = len(margrietslice2a.designs)

In [18]:
pickle.dump(margrietslice2a, open( "margrietslice2a.p", "wb" ) )

In [21]:
margrietslice2a = pickle.load( open("C:/Users/Admin_2/Documents/Thesis/LUS/DeltaScenarios_TUdelft1/margrietslice2a.p", "rb" )) 

In [26]:
len(margrietslice2a.designs)

350

In [19]:
margrietslice2b = copy.deepcopy(scenarios2000)
margrietslice2b.designs = margrietslice2b.designs[350:700]
margrietslice2b.n = len(margrietslice2b.designs)

In [20]:
pickle.dump(margrietslice2b, open( "margrietslice2b.p", "wb" ) )

In [15]:
margrietslice2b = pickle.load( open("C:/Users/Admin_2/Documents/Thesis/LUS/DeltaScenarios_TUdelft1/margrietslice2b.p", "rb" )) 

In [25]:
len(margrietslice2b.designs)

350

In [21]:
results = perform_experiments(LUSmodel, margrietslice2a)
experiments, outcomes = results

[MainProcess/INFO] performing 350 scenarios * 1 policies * 1 model(s) = 350 experiments
[MainProcess/INFO] performing experiments sequentially


C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesi

[MainProcess/INFO] 35 cases completed


C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesi

[MainProcess/INFO] 70 cases completed


...

[MainProcess/INFO] 280 cases completed


C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesi

[MainProcess/INFO] 315 cases completed


C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesi

[MainProcess/INFO] 350 cases completed
[MainProcess/INFO] experiments finished


In [22]:
save_results(results, "slice2A_Margrietexperiments2000.tar.gz")

[MainProcess/INFO] results saved successfully to C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1\slice2A_Margrietexperiments2000.tar.gz


In [30]:
results = perform_experiments(LUSmodel, margrietslice2b)
experiments, outcomes = results

[MainProcess/INFO] performing 350 scenarios * 1 policies * 1 model(s) = 350 experiments
[MainProcess/INFO] performing experiments sequentially


C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesi

[MainProcess/INFO] 35 cases completed


C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesi

[MainProcess/INFO] 70 cases completed


...

[MainProcess/INFO] 280 cases completed


C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesi

[MainProcess/INFO] 315 cases completed


C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesi

[MainProcess/INFO] 350 cases completed
[MainProcess/INFO] experiments finished


In [31]:
save_results(results, "slice2B_Margrietexperiments2000.tar.gz")

[MainProcess/INFO] results saved successfully to C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1\slice2B_Margrietexperiments2000.tar.gz


# Generate middle maps

In [14]:
def LUS_generatemodemaps(TigrisXLfile, inbreiding, Density, Claimfile, natureclaim, recvalue, restrictionnatureweight, naturepolicymap, policyweightnature, spreidingratio, valuenaturelocation, pumping, agrclaim):
    
    changeresidentialclaim(TigrisXLfile, inbreiding, Density)
    changeworkclaim(TigrisXLfile, Claimfile)
    changenatureclaim(natureclaim)
    changerecreationclaim(recvalue)
    changeagricultureclaim(agrclaim)
    
    naturesuitability(valuenaturelocation, policyweightnature, pumping)
    changehousingsuitability(spreidingratio)
    changenaturepolicy(naturepolicymap, restrictionnatureweight)
  
    
    
    #Run GeoDMS software:
    config = notebook_dir+'/PD/cfg/stam.dms'
    tree = '/EndResults/PerScenTimeUnit/STOOM_Y2012_Y2050/Run1'

    geodmsrun(config, tree)
    
    # Safe resulting land use map:
    src = notebook_dir+'/PD/cfg/stam/results/run1.asc'
    
    # Define output: 
    lusmap = np.loadtxt(src, skiprows=6)
    
    # Define dummy output:
    dummy = np.max(lusmap)
    
    return lusmap

In [15]:
#Cluster 1
middlemap_cluster1 = LUS_generatemodemaps(4, False, True, False, 3, False, True, 2, 3, True, 1, True, True)

C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1


In [16]:
#cluster 2
middlemap_cluster2 = LUS_generatemodemaps(1, True, True, True, 3, False, False, 3, 3, False, 1, False, True)

C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1


In [17]:
#Cluster 3
middlemap_cluster3 = LUS_generatemodemaps(1, True, True, True, 3, True, False, 1, 2, False, 3, True, True)

C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1


In [18]:
#Cluster 4
middlemap_cluster4 = LUS_generatemodemaps(3, True, False, False, 3, False, False, 3, 3, False, 1, False, True)

C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1


In [19]:
#Cluster 5
middlemap_cluster5 = LUS_generatemodemaps(2, False, True, True, 3, False, True, 2, 4, True, 2, True, True)

C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1


In [20]:
#Cluster 6
middlemap_cluster6 = LUS_generatemodemaps(1, False, False, True, 2, False, True, 2, 4, True, 2, True, True)

C:\Program Files\ObjectVision\GeoDms7212
C:\Users\Admin_2\Documents\Thesis\LUS\DeltaScenarios_TUdelft1
